<a href="https://colab.research.google.com/github/dfieman/CRNC/blob/main/LandslideConcFinder_v2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from scipy import interpolate, LowLevelCallable
import scipy.integrate as integrate
import multiprocessing
import sys
import datetime
from functools import partial
import os
import glob
import ctypes
import numba
from numba import jit
import itertools
#import more_itertools as mit
from scipy.interpolate import Rbf
from scipy.interpolate import griddata
import random
from matplotlib.lines import Line2D
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#############
# FUNCTIONS #
#############

# Open rasters as arrays
def gdal_open(DEM_name,folder):
    try:
        raster = gdal.Open(folder+DEM_name)
        nodata = raster.GetRasterBand(1).GetNoDataValue()
        array = raster.GetRasterBand(1).ReadAsArray()
        array[array==nodata]=np.nan
        return array
    except:
        print("Could not open:",folder+DEM_name)
        return None


# Slices the landslide depth every D50 until values in array equal 0. Rounds surface so divisible by the slicer length

# Slices the landslide depth every D50 until values in array equal 0. Rounds surface so divisible by the slicer length
def depthSlicer(LandslideArray,D50):
    depth = [LandslideArray.copy()]
    while np.any(LandslideArray >= 0.0):
        LandslideArray = np.round(LandslideArray/D50)*D50 #rounds so divisible by D50
        positive_values = LandslideArray > 0.0
        LandslideArray[positive_values] -= D50
        depth.append(LandslideArray.copy())#copy required due to odd behaviour
        LandslideArray[LandslideArray==0.0]=np.nan #so to not have a zero value more than once
    return depth

# PRODUCTION FUNCTIONS #PRES

#Convert elevation to units of hPa and g/cm^2. Input h = elevation [m] 2D array. Outputs pressure in hPa and g/cm2
def h_units(h):
    #Constants for pressure
    Psl = 1013.25 #hPa; atmopheric pressure at sea level
    tempsl = 288.15 #K; temperature at sea level
    dTdz = 0.0065 #K/m; adiabatic lapse rate
    g = 9.80665 #m/s^2
    air = 0.0289644 #kg/mol; molecular mass of air
    gas = 8.31446 #J/mol/K; gas constant
    baro = Psl * np.exp(-1*(g*air/(gas*dTdz))*(np.log(tempsl)-np.log(tempsl-(dTdz*h))))
    atPres = 1.019716*(1013.25-baro)
    return baro, atPres

## Scaling production functions:

#Spallation scaling production equations from Stone 2000:
#Input argument l =latitude [deg], pressure_array = elevation [hPa], and Pref = reference production rate at SLHL [atoms/g/yr]
def spalProd(l,pressure_array,CRN):
    #Interpolation for latitute coefficient. See Table 1 Stone 2000
    coeff = np.array([(31.8518, 250.3193, -0.083393, 7.4260E-5, -2.2397E-8),(34.3699, 258.4759, -0.089807, 7.9457E-5, -2.3697E-8),(40.3153, 308.9894, -0.106248, 9.4508E-5, -2.8234E-8),(42.0983, 512.6857, -0.120551, 1.1752E-4, -3.8809E-8), (56.7733, 649.1343, -0.160859, 1.5463E-4, -5.033E-8),(69.0720, 832.4566, -0.199252, 1.9391E-4, -6.3653E-8),(71.8733, 863.1927, -0.207069, 2.0127E-4, -6.6043E-8)])
    a_coeff = coeff[:,0]
    b_coeff = coeff[:,1]
    c_coeff = coeff[:,2]
    d_coeff = coeff[:,3]
    e_coeff = coeff[:,4]
    lat = np.array([0,10,20,30,40,50,60])
    a_func = interpolate.interp1d(lat,a_coeff)
    b_func = interpolate.interp1d(lat,b_coeff)
    c_func = interpolate.interp1d(lat,c_coeff)
    d_func = interpolate.interp1d(lat,d_coeff)
    e_func = interpolate.interp1d(lat,e_coeff)
    Sp = a_func(l)+(b_func(l)*np.exp(-1*pressure_array/150))+(c_func(l)*pressure_array)+(d_func(l)*(pressure_array**2))+(e_func(l)*(pressure_array**3)) #Scaling factor
    Pref = [3.84,25.92,11.7]
    return 0.978*Sp*Pref[CRN]

#Muon scaling production equations from Balco2017
@jit(nopython = True)
def muProd(baro,CRN):
    Pref = [0.0735,0.6764,3.067] #atoms/g/yr
    atten = [299.2,288.0,267.8] #hPa
    return Pref[CRN]*np.exp((1013.25-baro)/atten[CRN])

# Calculate the concentration of muons with depth using a pre-calculated grid of concentrations with depths and erosion rates
# from Heisinger scheme if cell was at steady-state
# note: this only considers the average pressure of the catchment to simplify
def CmuDepthSteadyState(erosion_array,depth):
    valid_values = depth > 0.0
    rows,cols = np.where(valid_values)

    erosion = np.where(np.isnan(depth),np.nan,erosion_array)
    erosion_flat = erosion.flatten()[~np.isnan(erosion.flatten())]
    depth_flat = depth.flatten()[~np.isnan(depth.flatten())]

    Cmu = griddata((Emu_grid.flatten(),Depth_grid.flatten()),Cmu_steadystate.flatten(),(erosion_flat,depth_flat),method='nearest')

    Cmu2D = np.full_like(depth,np.nan,dtype='float')
    for i,j,val in zip(rows,cols,Cmu2D):
        Cmu2D[i][j] = val

    return Cmu2D

# Calculates concentration with depth using pre-calculated grid  at the landslide reccurence interval
# from Heisinger scheme if cell had landslide
def CmuDepthLandslide(erosion_array,depth):
    valid_values = depth > 0.0
    rows,cols = np.where(valid_values)

    erosion = np.where(np.isnan(depth),np.nan,erosion_array)
    erosion_flat = erosion.flatten()[~np.isnan(erosion.flatten())]
    depth_flat = depth.flatten()[~np.isnan(depth.flatten())]

    Cmu = griddata((Emu_grid.flatten(),Depth_grid.flatten()),Cmu_landslide.flatten(),(erosion_flat,depth_flat),method='nearest')

    Cmu2D = np.full_like(depth,np.nan,dtype='float')
    for i,j,val in zip(rows,cols,Cmu2D):
        Cmu2D[i][j] = val

    return Cmu2D

#Muon attenuation length with elevation and erosion rate using pre-calculated grid
##note: griddate cannot handle nan values as inputs so need to get rid of them then replace back into 2d array
def LeffCalculator(pressure_array,erosion_array,P_grid,E_grid,Leff):

    valid_values = erosion_array>0.0
    rows,cols = np.where(valid_values)

    baro = np.where(np.isnan(erosion_array),np.nan,pressure_array)
    baro_flat = baro.flatten()[~np.isnan(baro.flatten())]
    erosion_flat = erosion_array.flatten()[~np.isnan(erosion_array.flatten())]
    #print(len(baro_flat),len(erosion_flat))
    Leffmu = griddata((P_grid.flatten(),E_grid.flatten()),Leff.flatten(),(baro_flat,erosion_flat),method='nearest')

    Leff2D = np.full_like(baro,np.nan,dtype='float')
    for i,j,val in zip(rows,cols,Leffmu):
        Leff2D[i][j] = val

    return Leff2D

#Concentration if t is inf and at the surface. Exponenital for muon and spallations
def surfaceConcentration(pressure_array,erosion_array,landslide_array):
    Psp = spalProd(latitude,pressure_array,CRN)
    #print(np.nanmean(Psp))
    Pmu = muProd(pressure_array,CRN)
    #print(np.nanmean(Pmu))
    Leffmu = LeffCalculator(pressure_array,erosion_array,P_grid,E_grid,Leff[CRN])
    Csp = (Psp/(decayConst[CRN]+(rho*erosion_array/sp_efold)))*np.exp(-rho*landslide_array/sp_efold)
    Cmu = (Pmu/(decayConst[CRN]+(rho*erosion_array/Leffmu)))*np.exp(-rho*landslide_array/Leffmu)
    #print(np.nanmean(Csp))
    #print(np.nanmean(Cmu))
    return Csp+Cmu

# Concentration with depth if the concentration was at steady-state
@jit(nopython = True)
def steadyStateDepth(erosion_array,depth):
    Csp_depth = (Psp/(decayConst[CRN]+(rho*erosion_array/sp_efold)))*np.exp(-rho*depth/sp_efold)
    Cmu_depth = (Pmu/(decayConst[CRN]+(rho*erosion_array/Leffmu)))*np.exp(-rho*depth/Leffmu)
    #Cmu_depth = CmuDepthSteadyState(erosion_array,depth)
    return Csp_depth+Cmu_depth

# Concentration with depth if there is inheritance from a previous landslide
@jit(nopython = True)
def previousLandslideDepth(erosion_array,depth,C_inheritance):
    Csp_depth = (Psp/(decayConst[CRN]+(rho*erosion_array/sp_efold)))*np.exp(-rho*depth/sp_efold)*(1-np.exp(-1*(decayConst[CRN]+(rho*erosion_array/sp_efold))*landslide_reccurence))
    Cmu_depth = (Pmu/(decayConst[CRN]+(rho*erosion_array/Leffmu)))*np.exp(-rho*depth/Leffmu)*(1-np.exp(-1*(decayConst[CRN]+(rho*erosion_array/Leffmu))*landslide_reccurence))
    #Cmu_depth = CmuDepthLandslide(erosion_array,depth)
    return Csp_depth+Cmu_depth+C_inheritance

@jit(nopython = True)
def concThruTime(erosion_array,C_inheritance,time):
    Csp_landslide_time = ((Psp/(decayConst[CRN]+(rho*erosion_array/sp_efold)))*(1-np.exp(-1*(decayConst[CRN]+(rho*erosion_array/sp_efold))*time)))
    Cmu_landslide_time = ((Pmu/(decayConst[CRN]+(rho*erosion_array/Leffmu)))*(1-np.exp(-1*(decayConst[CRN]+(rho*erosion_array/Leffmu))*time)))
    return Csp_landslide_time+Cmu_landslide_time+C_inheritance

In [ ]:
# Function for calculating the concentration through time of n percent of hillslope cells and t residence time.
def LandslideTimeCalculator(pressure_array,erosion_array,landslide_array):

    C_list = []

    C_landslide_surface_list = []

    mins = []
    maxs = []
    q1 = []
    q3 = []
    medians = []
    means = []
    whishi =[]
    whislo = []

    #for total_time:

    #Solve for initial steady-state surface concentration
    start = datetime.datetime.now()
    Csp_0_surface = (Psp/(decayConst[CRN]+(rho*erosion_array/sp_efold)))
    Cmu_0_surface = (Pmu/(decayConst[CRN]+(rho*erosion_array/Leffmu)))
    C_0_surface = Csp_0_surface + Cmu_0_surface
    #print('Initial surface C:',C_0_surface)
    C_0_surface_flat = C_0_surface[~np.isnan(C_0_surface)]
    ##Add extra arrays for residence time and multiple by catchment percent
    for residence in range(residence_time):
        C_0_surface_connected = np.random.choice(C_0_surface_flat,size=int(len(C_0_surface_flat)*catchment_percent),replace=False)
        C_list.append(C_0_surface_connected)
    end = datetime.datetime.now()
    print('calculated initial steady-state surface concentration took:',end-start)
    #print('C_list:',C_list)
    C_inheritance = np.zeros_like(C_0_surface) #initial inheritance
    C_previous_surface = C_0_surface #initiate background concentration

    landslide_time = range(t_interval,landslide_reccurence-t_interval+1,t_interval)
    total_time = np.zeros_like(C_0_surface) #Make time array
    landslide_masks = [np.full_like(C_0_surface,False,dtype=bool)] #initial landslide mask
    Csp_0 = np.zeros_like(C_0_surface)
    Cmu_0 = np.zeros_like(C_0_surface)

    for i in range(0,len(landslide_array)):
        print('On landslide number:',i+1)
        #print('C_previous surface:',C_previous_surface)
        #print('C_inheritance',C_inheritance)
        current_landslide_mask = ~np.isnan(landslide_array[i])
        #print('current landslide mask:',current_landslide_mask)
        total_time = np.where(current_landslide_mask,0,total_time) #start time over where there was a landslide
        #print('total_time:',total_time)
        #landslide_masks.append(current_landslide_mask)
        previous_landslide_masks = np.logical_or.reduce(landslide_masks) #Combine all landslide masks
        #print('previous landslide masks:',previous_landslide_masks)
        #Calculate the landslide concentration with depth
        start = datetime.datetime.now()
        depth_arrays = depthSlicer(landslide_array[i],D50)
        end = datetime.datetime.now()
        print('depth slicer took:',end-start)

        #Calculate the concentration of each depth
        C_total_depths = []
        for depth in depth_arrays:
            nan_mask = ~np.isnan(depth)
            C_each_depth = np.where(previous_landslide_masks,previousLandslideDepth(erosion_array,depth,C_inheritance),steadyStateDepth(erosion_array,depth))
            #print('Each depth:',C_each_depth)
            C_total_depths.append(C_each_depth[nan_mask]) #get rid of nans

        #memory management
        depth_arrays = None

        print('Calculated the concentration of each landslide depth')
        C_total_depths_merged = list(itertools.chain(*C_total_depths)) #combine all depth concentrations
        print('Mean landslide concentration:',np.nanmean(C_total_depths_merged))
        #print('Total depths merged:',C_total_depths_merged)
        #Make surface concentration where there wasn't landslides then add that to C_list
        C_landslide_surface = C_previous_surface[np.isnan(landslide_array[i])]
        #print('C_landslide_surface:',C_landslide_surface)
        C_landslide_surface_connected = np.random.choice(C_landslide_surface,size=int(len(C_landslide_surface)*catchment_percent),replace=False)
        C_list.append(np.concatenate((C_total_depths_merged,C_landslide_surface_connected),axis=None)) #combine surface and landsdie depths
        #print('length of C_list:',len(C_list))

        #Make the new current surface
        C_max_depth = np.where(previous_landslide_masks,previousLandslideDepth(erosion_array,landslide_array[i],C_inheritance),steadyStateDepth(erosion_array,landslide_array[i]))
        C_current_surface = np.where(current_landslide_mask,C_max_depth,C_previous_surface)
        #print('C_current_surface:',C_current_surface)
        C_inheritance = np.where(C_max_depth>0,C_max_depth,C_inheritance) #Make inheritance where there was a landslide
        #print('New C_inheritance:',C_inheritance)

        #Calculate current landslide surface through time:
        for time in landslide_time:
            print('On time step:',time)
            total_time = np.where(previous_landslide_masks | current_landslide_mask,total_time+t_interval,total_time) #Array for each time step. Time starts over for new landslide
            time_mask = total_time>0
            C_time_step = np.where(time_mask,concThruTime(erosion_array,C_inheritance,total_time),C_current_surface)
            #print('C each time step:',C_time_step)
            C_time_step_connected = np.random.choice(C_time_step.flatten(),size=int(len(C_time_step.flatten())*catchment_percent),replace=False)
            C_list.append(C_time_step_connected)

        #Add current landslide to previous landslides
        landslide_masks.append(current_landslide_mask)
        previous_landslide_masks = np.logical_or.reduce(landslide_masks)
        #Make surface where next landslide starts
        C_previous_surface =  np.where(time_mask,concThruTime(erosion_array,C_inheritance,landslide_reccurence),C_current_surface)

    #Add sediment mixing by combining the last n arrays
    print("Loop length of %s"%(len(C_list)+1-residence_time))
    for i in range(0,len(C_list)+1-residence_time):
        print("At %s"%(i))
        combined_residence_C = C_list[i:i+residence_time]
        concantenated_arrays = np.concatenate(combined_residence_C)

        min_value = np.nanmin(concantenated_arrays)
        max_value = np.nanmax(concantenated_arrays)
        median_value = np.nanmedian(concantenated_arrays)
        mean_value = np.nanmean(concantenated_arrays)
        q1_value = np.nanquantile(concantenated_arrays,0.25)
        q3_value = np.nanquantile(concantenated_arrays,0.75)
        hi = min(max_value,q3+((q3-q1)*1.5))
        lo = max(min_value,q1-((q3-q1)*1.5))

        mins.append(min_value)
        maxs.append(max_value)
        medians.append(median_value)
        means.append(mean_value)
        q1.append(q1_value)
        q3.append(q3_value)
        whishi.append(hi)
        whislo.append(lo)

        #memory management
        concatenated_arrays = None

    return mins,maxs,q1,q3,medians,means,whishi,whislo

In [ ]:
# Function for calculating the concentration of the landslide cells only through time for ONE landslide
# # (use this function if want to know the difference in concentration with a landslide between uniform and slope dependent)
def LandslideOnlyTime(pressure_array,erosion_array,landslide_array):
    C_list = []

    mins = []
    maxs = []
    q1 = []
    q3 = []
    medians = []
    means = []
    whishi = []
    whislo = []

    landslide_mask = np.isnan(landslide_array)
    pressure_array = np.where(landslide_mask,np.nan,pressure_array)
    erosion_array = np.where(landslide_mask,np.nan,erosion_array)

    #Solve for initial steady-state surface concentration
    start = datetime.datetime.now()
    Csp_0_surface = (Psp/(decayConst[CRN]+(rho*erosion_array/sp_efold)))
    Cmu_0_surface = (Pmu/(decayConst[CRN]+(rho*erosion_array/Leffmu)))
    C_0_surface = Csp_0_surface + Cmu_0_surface
    print('Mean surface C',np.nanmean(C_0_surface))
    C_list.append(C_0_surface)
    #print('Initial surface C:',C_0_surface)
    #total_time = np.zeros_like(C_0_surface)
    landslide_time = range(t_interval,landslide_reccurence-t_interval+1,t_interval)
    depth_arrays = depthSlicer(landslide_array,D50)
    end = datetime.datetime.now()
    #print('depth slicer took:',end-start)

    #Calculate the concentration of each depth
    C_total_depths = []
    for depth in depth_arrays:
        nan_mask = ~np.isnan(depth)
        C_each_depth = steadyStateDepth(erosion_array,depth)
        print('Each depth:',C_each_depth)
        C_total_depths.append(C_each_depth[nan_mask]) #get rid of nans

    #print('Calculated the concentration of each landslide depth')
    C_total_depths_merged = list(itertools.chain(*C_total_depths)) #combine all depth concentrations
    #print('Mean landslide depth',np.nanmean(C_total_depths_merged))
    C_list.append(C_total_depths_merged)

    depth_arrays = None
    #Calculate current landslide surface through time:
    C_max_depth = steadyStateDepth(erosion_array,landslide_array)
    print('C amx depth',np.nanmean(C_max_depth))
    for time in landslide_time:
        print('On time step:',time)
        C_time_step = concThruTime(erosion_array,C_max_depth,time)
        print('Mean each time step',np.nanmean(C_time_step))
        C_list.append(C_time_step)


    counter = 0
    for c in C_list:
        counter+=1
        print(counter)
        min_c = np.nanmin(c)
        max_c = np.nanmax(c)
        median = np.nanmedian(c)
        mean = np.nanmean(c)
        q1_c = np.nanquantile(c,0.25)
        q3_c = np.nanquantile(c,0.75)
        hi = min(max_c,q3+((q3-q1)*1.5))
        lo = max(min_c,q1-((q3-q1)*1.5))
        mins.append(min_c)
        maxs.append(max_c)
        q1.append(q1_c)
        q3.append(q3_c)
        medians.append(median)
        means.append(mean)
        whishi.append(hi)
        whislo.append(lo)


    return mins,maxs,q1,q3,medians,means,whishi,whislo

In [ ]:
# Function for calculating the concentration in the channel through time only considering the landslide cells as input in every time step
def LandslideOnlyTimeCalculator(pressure_array,erosion_array,landslide_array):
    C_list = []

    C_landslide_surface_list = []

    mins = []
    maxs = []
    q1 = []
    q3 = []
    medians = []
    means = []
    whishi = []
    whislo = []

    #for total_time:

    #Solve for initial steady-state surface concentration
    start = datetime.datetime.now()
    Csp_0_surface = (Psp/(decayConst[CRN]+(rho*erosion_array/sp_efold)))
    Cmu_0_surface = (Pmu/(decayConst[CRN]+(rho*erosion_array/Leffmu)))
    C_0_surface = Csp_0_surface + Cmu_0_surface
    #print('Initial surface C:',C_0_surface)
    C_0_surface_flat = C_0_surface[~np.isnan(C_0_surface)]
    ##Add extra arrays for residence time and multiple by catchment percent

    for residence in range(residence_time):
        C_0_surface_connected = np.random.choice(C_0_surface_flat,size=int(len(C_0_surface_flat)*catchment_percent),replace=False)
        C_list.append(C_0_surface_connected)
    end = datetime.datetime.now()
    print('calculated initial steady-state surface concentration took:',end-start)
    #print('C_list:',C_list)

    C_inheritance = np.zeros_like(C_0_surface) #initial inheritance
    C_previous_surface = C_0_surface #initiate background concentration

    landslide_time = range(t_interval,landslide_reccurence-t_interval+1,t_interval)
    total_time = np.zeros_like(C_0_surface) #Make time array
    landslide_masks = [np.full_like(C_0_surface,False,dtype=bool)] #initial landslide mask
    Csp_0 = np.zeros_like(C_0_surface)
    Cmu_0 = np.zeros_like(C_0_surface)

    for i in range(0,len(landslide_array)):
        print('On landslide number:',i+1)
        #print('C_previous surface:',C_previous_surface)
        #print('C_inheritance',C_inheritance)
        current_landslide_mask = ~np.isnan(landslide_array[i])
        #print('current landslide mask:',current_landslide_mask)
        total_time = np.where(current_landslide_mask,0,total_time) #start time over where there was a landslide
        #print('total_time:',total_time)
        #landslide_masks.append(current_landslide_mask)
        previous_landslide_masks = np.logical_or.reduce(landslide_masks) #Combine all landslide masks
        #print('previous landslide masks:',previous_landslide_masks)
        #Calculate the landslide concentration with depth
        start = datetime.datetime.now()
        depth_arrays = depthSlicer(landslide_array[i],D50)
        end = datetime.datetime.now()
        print('depth slicer took:',end-start)

        #Calculate the concentration of each depth
        C_total_depths = []
        for depth in depth_arrays:
            nan_mask = ~np.isnan(depth)
            C_each_depth = np.where(previous_landslide_masks,previousLandslideDepth(erosion_array,depth,C_inheritance),steadyStateDepth(erosion_array,depth))
            #print('Each depth:',C_each_depth)
            C_total_depths.append(C_each_depth[nan_mask]) #get rid of nans

        #memory management
        depth_arrays = None

        print('Calculated the concentration of each landslide depth')
        C_total_depths_merged = list(itertools.chain(*C_total_depths)) #combine all depth concentrations
        print('Mean landslide concentration:',np.nanmean(C_total_depths_merged))
        #print('Total depths merged:',C_total_depths_merged)

        #Make the new current surface
        C_max_depth = np.where(previous_landslide_masks,previousLandslideDepth(erosion_array,landslide_array[i],C_inheritance),steadyStateDepth(erosion_array,landslide_array[i]))
        C_current_surface = np.where(current_landslide_mask,C_max_depth,C_previous_surface)
        #print('C_current_surface:',C_current_surface)
        C_inheritance = np.where(C_max_depth>0,C_max_depth,C_inheritance) #Make inheritance where there was a landslide
        #print('New C_inheritance:',C_inheritance)

        #Calculate current landslide surface through time. Cells contributing into the channel are only cells that have had a landslide:
        for time in landslide_time:
            print('On time step:',time)
            total_time = np.where(previous_landslide_masks | current_landslide_mask,total_time+t_interval,total_time) #Array for each time step. Time starts over for new landslide
            time_mask = total_time>0
            C_time_step = np.where(time_mask,concThruTime(erosion_array,C_inheritance,total_time),C_current_surface)
            #print('C each time step:',C_time_step)
            C_time_step_connected = np.where(current_landslide_mask,C_time_step,np.nan)
            C_list.append(C_time_step_connected[~np.isnan(C_time_step_connected)])

        #Add current landslide to previous landslides
        landslide_masks.append(current_landslide_mask)
        previous_landslide_masks = np.logical_or.reduce(landslide_masks)
        #Make surface where next landslide starts
        C_previous_surface =  np.where(time_mask,concThruTime(erosion_array,C_inheritance,landslide_reccurence),C_current_surface)

    #Add sediment mixing by combining the last n arrays
    print("Loop length of %s"%(len(C_list)+1-residence_time))
    for i in range(0,len(C_list)+1-residence_time):
        print("At %s"%(i))
        combined_residence_C = C_list[i:i+residence_time]
        concantenated_arrays = np.concatenate(combined_residence_C)

        min_value = np.nanmin(concantenated_arrays)
        max_value = np.nanmax(concantenated_arrays)
        median_value = np.nanmedian(concantenated_arrays)
        mean_value = np.nanmean(concantenated_arrays)
        q1_value = np.nanquantile(concantenated_arrays,0.25)
        q3_value = np.nanquantile(concantenated_arrays,0.75)
        hi = min(max_value,q3+((q3-q1)*1.5))
        lo = max(min_value,q1-((q3-q1)*1.5))

        mins.append(min_value)
        maxs.append(max_value)
        medians.append(median_value)
        means.append(mean_value)
        q1.append(q1_value)
        q3.append(q3_value)
        whishi.append(hi)
        whislo.append(lo)

        #memory management
        concatenated_arrays = None

    return mins,maxs,q1,q3,medians,means,whishi,whislo

In [ ]:
if __name__ == '__main__':

    ######################################
    # Paramters for production functions #
    ######################################

    CRNs = ['Be10','Al26','C14']
    halfLife = [1.36e6, 7.17e5, 5730] #Half life
    decayConst = np.log(2)/halfLife #Decay constant
    fc = [0.704, 0.296, 0.704] #chemical compound factor (Heisinger 2002b)
    fd = [0.1828, 0.6559, 0.1828] #fraction of muons stopped by oxygen and absorbed by the nucleus before decay of muon (Heisinger 2002b)
    fstar = [0.0043, 0.022, 0.137] #Probability for particle emission to the radionuclide (Heisinger 2002b)
    sigmaGeV = [8.6e-29/(190**0.75), 1.41e-27/(190**0.75), 0.45e-27/(190**0.75)] # Muon interaction cross-section for the reaction that produces nuclide i (Table 1 Heisinger 2002a). 10Be updated in v2.2 Balco 2008
    Nnuclei = [2.006e22, 1.003e22, 2.006e22] #number density of atoms in target element
    sp_efold = 160 #effective attenuation length into rock [g/cm2]

    #For indexing
    Be10 = 0
    Al26 = 1
    C14 = 2
    slope_depend = 0
    uniform = 1
    all_landslides = 0
    connected = 1
    #reference production rate in atoms/g/yr for spallation production. For 10Be we use number from Putnam et al, C14 from Schaefer et al 2014. We use a 6.75 ratio for 26/10.
    Pref = [3.84,25.92, 11.7]
    Pref_unc = [0.08,0.08,0.9]

    ##############
    #USER INPUTS:#
    ##############

    ## Which CRN would you like to model?
    CRN = Be10 #Option are 'Be10','Al26', or 'C14'. Can only do one at a time.

    # ##Which erosion type would you like to model?
    eType = slope_depend #options are 'uniform, 'slope_depend' npo quotes please

    ## Percent of the catchment is contributing to the river?
    catchment_percents = [0.95,0.5,0.05]

    ## Maximum run time (years)?
    tmax = 5

    ## Residence time of sediment in the river? (years)
    residence_times = [1,5,10]

    ## Landslide reccurence interval (years)
    landslide_reccurence = 500

    ## Time interval model runs in (years)
    t_interval = 1

    ## Would you like all landslides or connected landslides?
    landslideType = all_landslides #options are all_landslides or connected

    rho = 2.65 #rock density [g/cm2]
    latitude = 42 #[deg]
    D50 = 100 #the D50 to slice the landslide depth [cm]

    #DEM_name = 'Hapuku'
    DEM_name = 'Kowhai'
    print(DEM_name)
    # #Where rasters are saved
    #main_folder = '/Users/home/fiemandi/CRNC/'
    #elevation_folder = '/Users/home/fiemandi/CRNC/Topo_Analysis/'
    #erosion_folder = '/Users/home/fiemandi/CRNC/Surface_Conc_Erate_Maker/'

    #elevation_folder = r'C:\\Users\\cbradbury\\Documents\\Topo_Analysis\\'
    #erosion_folder = r'C:\\Users\\cbradbury\\Documents\\Erosion_Rasters\\'

    elevation_folder = '/content/drive/MyDrive/Topo_Analysis/'
    erosion_folder = '/content/drive/MyDrive/Erosion_Rasters/'
    landslide_folder = '/content/drive/MyDrive/Model Landslides/'
    rasterformat = '.tif'
    uniform_erosion = 0.246 if DEM_name == 'Hapuku' else 0.288

Kowhai


In [ ]:
    if DEM_name == 'Kowhai':
        landslide_rasters = ['Kowhai_ModelLandslides_0','Kowhai_ModelLandslides_1','Kowhai_ModelLandslides_2','Kowhai_ModelLandslides_3','Kowhai_ModelLandslides_4','Kowhai_ModelLandslides_5','Kowhai_ModelLandslides_6','Kowhai_ModelLandslides_7','Kowhai_ModelLandslides_8','Kowhai_ModelLandslides_9']
    if DEM_name == 'Hapuku':
        landslide_rasters = ['Hapuku_ModelLandslides_0','Hapuku_ModelLandslides_1','Hapuku_ModelLandslides_2','Hapuku_ModelLandslides_3','Hapuku_ModelLandslides_4','Hapuku_ModelLandslides_5','Hapuku_ModelLandslides_6','Hapuku_ModelLandslides_7','Hapuku_ModelLandslides_8','Hapuku_ModelLandslides_9']
    multiple_distributions = [gdal_open(landslide_raster+rasterformat,landslide_folder)*100 for landslide_raster in landslide_rasters]

In [ ]:
    # #Open rasters to convert to arrays and covert erosion and depth from m to cm
    erosion_raster = 'Hapuku_K0.003_Sc9.89_SlopeErosion' if DEM_name == 'Hapuku' else 'Kowhai_K0.004_Sc9.90_SlopeErosion'
    landslide_rasters = DEM_name+'_SourceAreas'#,DEM_name+'_SourceAreas_Con']
    elevation_raster = f'{DEM_name}_Masked_Wshd'
    print(erosion_raster)
    print(landslide_rasters)
    print(elevation_raster)

    check_file = os.path.exists(elevation_folder+landslide_rasters+rasterformat)
    print(check_file)

    KaikouraEQ_landslideArray = gdal_open(landslide_rasters+rasterformat,elevation_folder)*100

    elevationArray = gdal_open(elevation_raster+rasterformat,elevation_folder)

    erosionArrays = [gdal_open(erosion_raster+rasterformat,erosion_folder),np.where(np.isnan(elevationArray),np.nan,np.full_like(elevationArray,uniform_erosion))]
    erosionArray = erosionArrays[eType]
    #erosionArray = np.where(np.isnan(elevationArray),np.nan,np.full_like(elevationArray,uniform_erosion))
    #print(len(elevationArray.flatten()[~np.isnan(elevationArray.flatten())]),len(erosionArray.flatten()[~np.isnan(erosionArray.flatten())]))

    print('opened all input rasters')

    #Pre-calculated Leff grid from Balco 2016 for muon efolding length
    #Erate in cm/yr
    Erate = [0,0.0001,0.000158489,0.000251189,0.000398107,0.000630957,0.001,0.001584893,0.002511886,0.003981072,0.006309573,0.01,0.015848932,0.025118864,0.039810717,0.063095734,0.1,0.158489319,0.251188643,0.398107171,0.630957344,1]
    Pressure = [450,480,510,540,570,600,630,660,690,720,750,780,810,840,870,900,930,960,990,1020]

    P_grid,E_grid = np.meshgrid(Pressure,Erate)

    #Rows are erosion rate, columns are pressure
    Leff26 = np.genfromtxt(elevation_folder+'Leff26.csv',delimiter=',')
    Leff10 = np.genfromtxt(elevation_folder+'Leff10.csv',delimiter=',')
    #Leff26 = np.genfromtxt(r'C:\\Users\\cbradbury\\Documents\\Topo_Analysis\\Leff26.csv',delimiter=',')
    #Leff10 = np.genfromtxt(r'C:\\Users\\cbradbury\\Documents\\Topo_Analysis\\Leff10.csv',delimiter=',')
    Leff = [Leff10,Leff26,Leff10] #Greg didn't have the C14 grid so assuming it is equal to Be10 for now

Kowhai_K0.004_Sc9.90_SlopeErosion
Kowhai_SourceAreas
Kowhai_Masked_Wshd
True
opened all input rasters


In [ ]:
    ################
    # Calculations #
    ################

    #Convert elevation [m] (2D array) barometric pressure [hPa] and atmospheric pressure [g/cm2]
    pressureArray, atDepth = h_units(elevationArray)

    landslideArrays = KaikouraEQ_landslideArray
    # pressureArray = np.array([[100,100,100],[200,200,200],[300,300,300]])
    # erosionArray = np.array([[0.1,0.1,0.1],[0.01,0.01,0.01],[0.01,0.01,0.01]])
    # landslideArray = [np.array([[300,300,np.nan],[300,300,100],[np.nan,np.nan,100]]),np.array([[400,400,np.nan],[np.nan,np.nan,np.nan],[200,200,100]])]

    #Solve for surface spallation and muon production:
    #Psp = spalProd(latitude,pressureArray,CRN)
    #Pmu = muProd(pressureArray,CRN)
    #Leffmu = LeffCalculator(pressureArray,erosionArray,P_grid,E_grid,Leff[CRN])
    print('solved for production stuff')

solved for production stuff


In [ ]:
    total_catchment_cells = np.count_nonzero(elevationArray[~np.isnan(elevationArray)],axis=None)
    total_landslide_cells = np.count_nonzero(landslideArrays[~np.isnan(landslideArrays)],axis=None)
    flux_5 = total_catchment_cells*0.05
    flux_50 = total_catchment_cells*0.5
    flux_95 = total_catchment_cells*0.95
    print(flux_5,flux_50,flux_95)
    print(total_landslide_cells)
    print(total_catchment_cells)

2759518.1 27595181.0 52430843.9
2608608
55190362


In [ ]:
    #"""
    #loading previously calculated muon data
    if DEM_name == "Hapuku":
        depth_grid = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Hapuku_Depths.csv',delimiter=',')
        erosion_grid = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Hapuku_Erosions.csv',delimiter=',')
        muon_grid = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Hapuku_SlopeDependent_Muons.csv',delimiter=',')
        muon_grid_LS = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Hapuku_SlopeDependent_Muons_LS.csv',delimiter=',')

    if DEM_name == "Kowhai":
        depth_grid = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Kowhai_Depths.csv',delimiter=',')
        erosion_grid = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Kowhai_Erosions.csv',delimiter=',')
        muon_grid = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Kowhai_SlopeDependent_Muons.csv',delimiter=',')
        muon_grid_LS = np.genfromtxt(r'/content/drive/MyDrive/Colab Notebooks/Kowhai_SlopeDependent_Muons_LS.csv',delimiter=',')
    #"""

In [ ]:
    mins,maxs,q1,q3,medians,means,whishi,whislo = LandslideOnlyTime(pressureArray,erosionArray,KaikouraEQ_landslideArray[0])
    time_list = [x*t_interval for x in range(0,len(mins),1)]
    C_data = {
        'Mins':mins,
        'Maxs': maxs,
        'Q1': q1,
        'Q3': q3,
        'Low_Whisk':whislo,
        'Up_Whisk':whishi,
        'Medians':medians,
        'Means':means,
        'Time':time_list
        }

    output_name = DEM_name+'_'+CRNs[CRN]+'_UniformE.csv' if eType == uniform else DEM_name+'_'+CRNs[CRN]+'_SlopeE.csv'
    pd.DataFrame(C_data).to_csv(output_name,index=False)#rename this a better name
    print("made csv")

Mean surface C 2866.137345850908
C amx depth 277.30091330806334
On time step: 1
Mean each time step 287.435977346812
On time step: 2
Mean each time step 297.5302605380225
On time step: 3
